# Web Scrapping Assignment- 3 

1. Write a python program which searches all the product under a particular product vertical
from www.amazon.in. The product verticals to be searched will be taken as input from user.
For e.g. If user input is ‘guitar’. Then search for guitars.

2. In the above question, now scrape the following details of each product listed in first 3 pages 
of your search results and save it in a data frame and csv. In case if any product vertical has 
less than 3 pages in search results then scrape all the products available under that product 
vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of 
Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" 
and “Product URL”. In case, if any of the details are missing for any of the product then 
replace it by “-“

In [23]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
import time
import re
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from datetime import datetime

In [27]:
driver=driver = webdriver.Chrome(r'C:\Users\Admin\Downloads\chromedriver_win32 (2)\chromedriver.exe')
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="695ba05f0e8952545779aa50f57a4546")>

In [28]:
driver.get("https://www.amazon.in/")

In [29]:
Search_Keys = input("Type the Keyword to Search on Amazon:----> ")

Type the Keyword to Search on Amazon:----> guitar


In [30]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@id='twotabsearchtextbox']"))).send_keys(Search_Keys)

In [31]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@id='nav-search-submit-text']"))).click()

# Now Lets Scrape The above mentioned Data.

In [32]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@id='nav-search-submit-text']"))).click()

In [33]:
URT1=[]
for i in range(0, 3):
    for i in driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']"):
        URT1.append(i.get_attribute('href'))

    driver.find_element_by_xpath("//li[@class='a-last']").click()
    time.sleep(3)
        
#Lets create required empty lists required for the 
Avg_Ratings=[]
Brand=[]
No_of_Ratings=[]
Price=[]
Return_Exchange=[]
Expected_Delivery=[]
Availability=[]
Other_Details=[]
Product_URL=[]

In [34]:
for url in URT1:
    driver.get(url)
    
#To find the Avg_Ratings of the product
    try:                  
        rate=driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']")
        rate.click()                                                      
        rating=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-base']")
        Avg_Ratings.append(rating.text)
        
    except NoSuchElementException   as e:
        Avg_Ratings.append("-")
               
#To find the Brand Name & Details of the product
    try: 
        brand=driver.find_element_by_xpath("//span[@id='productTitle']")
        Brand.append(brand.text)
        
    except NoSuchElementException   as e:
        Brand.append("-")       
        
#To find the Brand Name & Details of the product  
    try: 
        ratings=driver.find_element_by_xpath("//span[@class='a-size-base a-color-secondary']")
        No_of_Ratings.append(ratings.text)
        
    except NoSuchElementException   as e:
        No_of_Ratings.append("-")
        
        
#To find the Price of the product 
    try: 
        price=driver.find_element_by_xpath("//td[@class='a-span12']")
        Price.append(price.text)
        
    except NoSuchElementException   as e:
        Price.append("-")
              
#To find the Return & Exchange policy of the product 
    try: 
        
        ret_exchange=driver.find_element_by_xpath("//div[@class='a-column a-span3 a-text-center icon-container  icon-grid-width']")#locating the brand
        Return_Exchange.append(ret_exchange.text)
        
    except NoSuchElementException   as e:
        Return_Exchange.append("-")
              
#To find the Expected Delivery policy of the product 
    try: 
        
        Delivery=driver.find_element_by_xpath("//div[@id='dynamicDeliveryMessage']")
        Expected_Delivery.append(Delivery.text)
        
    except NoSuchElementException   as e:
        Expected_Delivery.append("-")        
        
#To find the Availability of the product 
    try: 
        
        availability=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-success']")
        Availability.append(availability.text)
        
    except NoSuchElementException   as e:
        Availability.append("-")       
        
#To find the Other Details of the product
    try: 
        
        Details=driver.find_element_by_xpath("//div[@id='feature-bullets']")
        Other_Details.append(Details.text)
        
    except NoSuchElementException   as e:
        Other_Details.append("-")

#Creating the dataframe for the Searched Keyword/Product searched on Amazon

DF=pd.DataFrame({})
DF['Brand']=Brand
DF['No_of_Ratings']=No_of_Ratings
DF['Avg_Ratings']=Avg_Ratings
DF['Price']=Price
DF['Expected_Delivery']=Expected_Delivery
DF['Other Details']=Other_Details


#Creating the dataframe for Product_Vertical
Product_Vertical=pd.DataFrame({})

#Brand Name of the Guitar
Product_Vertical['Brand Name'] = DF['Brand'].str.split(' ').str[0]

#Name Of Product/Guitar mentioned on Amazon
Product_Vertical['Name Of Product'] = DF['Brand'].str.split(' ').str[1:]
Product_Vertical['Name Of Product'] = DF['Brand'].str.replace('[.,(),]', '')

#Average Ratings for the Guitar 
Product_Vertical['Average Rating']=DF['Avg_Ratings'].str.split(' ').str[0]

#No_of_Ratings for the Guitar 
Product_Vertical['No Of Ratings'] = DF['No_of_Ratings'].str.split(' ').str[0]

#Price of the Guitar
Product_Vertical['Price_INR'] = DF['Price'].str.split(' ').str[1:2]

#Return policy of product
Product_Vertical['Return Policy'] = Return_Exchange

#Expected Delivery of product
Product_Vertical['Expected Delivery'] = DF['Expected_Delivery'].str.split(' ').str[2:5].replace('[.,(),]', '')

#Availability of product
Product_Vertical['Product Availability'] = Availability

#Other Details of product
Product_Vertical['Other Details'] = DF['Other Details'].str.replace('\n', ' ')

#URL of product
Product_Vertical['Product URL'] = URT1

print('The DataFrame for Searched product on https://www.amazon.in/ is :')

print("\n")
Product_Vertical

The DataFrame for Searched product on https://www.amazon.in/ is :




,Brand Name,Name Of Product,Average Rating,No Of Ratings,Price_INR,Return Policy,Expected Delivery,Product Availability,Other Details,Product URL
0,Juârez,Juârez Acoustic Guitar 38 Inch Cutaway 038C wi...,4,"13,211",[],-,"[Tuesday,, Oct, 5]",In stock.,"About this item Black Glossy Finish, Number of...",https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,Kadence Guitar Frontier Series Blue Acoustic G...,4,Total,[],-,"[Thursday,, Oct, 7]",In stock.,About this item Product Type : Acoustic Guitar...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Juarez,Juarez Acoustic Guitar Kit 38 Inch Cutaway 38C...,4,"4,330",[],-,"[Wednesday,, Oct, 6]",In stock.,About this item Number of Frets: 18 Acoustic g...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Juârez,Juârez JRZ38C/WH 6 Strings Acoustic Guitar 38 ...,3.9,485,[],-,"[Wednesday,, Oct, 6]",In stock.,"About this item Number of Frets: 18, Acoustic ...",https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,Juârez Acoustic Guitar 38 Inch Cutaway 038C wi...,4,"13,211",[],-,"[Tuesday,, Oct, 5]",In stock.,"About this item Black Glossy Finish, Number of...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
187,Blueberry,Blueberry B40 40 Inch Acoustic Guitar Kit inbu...,1,Total,[],-,"[Friday,, Oct, 8]",In stock.,"About this item Material: Top: Spruce, Side & ...",https://www.amazon.in/gp/slredirect/picassoRed...
188,Giuson,Giuson Venus Pinewood Acoustic Guitar With Bag...,-,Total,[],-,"[Saturday,, Oct, 9]",In stock.,"41 inch Cut-A-Way Acoustic Guitar, Fret board:...",https://www.amazon.in/gp/slredirect/picassoRed...
189,Giuson,Giuson Venus Rosewood Acoustic Guitar With Bag...,-,Total,[],-,"[Saturday,, Oct, 9]",In stock.,About this item 41 inch Cut-A-Way Acoustic Gui...,https://www.amazon.in/gp/slredirect/picassoRed...
190,Guitar,Guitar Nut Practical Headless Guitar Bridge Bl...,-,-,[],-,"[Oct, 29, -]",-,Professional and practical design with beautif...,https://www.amazon.in/gp/slredirect/picassoRed...


In [36]:
DF.to_csv('DF.csv')

In [37]:
df = pd.read_csv('DF.csv')
df

,Unnamed: 0,Brand,No_of_Ratings,Avg_Ratings,Price,Expected_Delivery,Other Details
0,0,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","13,211 global ratings",4 out of 5,"₹2,099.00","FREE delivery: Tuesday, Oct 5 Details\nFastest...","About this item\nBlack Glossy Finish, Number o..."
1,1,"Kadence Guitar Frontier Series, Blue Acoustic ...",Total price:,4 out of 5,"₹5,999.00","FREE delivery: Thursday, Oct 7 Details\nFastes...",About this item\nProduct Type : Acoustic Guita...
2,2,"Juarez Acoustic Guitar Kit, 38 Inch Cutaway, 3...","4,330 global ratings",4 out of 5,"₹2,299.00","FREE delivery: Wednesday, Oct 6",About this item\nNumber of Frets: 18\nAcoustic...
3,3,Juârez JRZ38C/WH 6 Strings Acoustic Guitar 38 ...,485 global ratings,3.9 out of 5,"₹2,399.00","FREE delivery: Wednesday, Oct 6","About this item\nNumber of Frets: 18, Acoustic..."
4,4,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","13,211 global ratings",4 out of 5,"₹2,099.00","FREE delivery: Tuesday, Oct 5 Details\nFastest...","About this item\nBlack Glossy Finish, Number o..."
...,...,...,...,...,...,...,...
187,187,"Blueberry B40, 40 Inch Acoustic Guitar Kit inb...",Total price:,1 out of 5,"₹4,080.00","FREE delivery: Friday, Oct 8 Details","About this item\nMaterial: Top: Spruce, Side &..."
188,188,Giuson Venus Pinewood Acoustic Guitar With Bag...,Total price:,-,"₹3,790.00","FREE delivery: Saturday, Oct 9 Details","41 inch Cut-A-Way Acoustic Guitar, Fret board:..."
189,189,Giuson Venus Rosewood Acoustic Guitar With Bag...,Total price:,-,"₹3,790.00","FREE delivery: Saturday, Oct 9 Details",About this item\n41 inch Cut-A-Way Acoustic Gu...
190,190,"Guitar Nut, Practical Headless Guitar Bridge, ...",-,-,"₹1,379.00",FREE delivery: Oct 29 - Nov 2 Details,Professional and practical design with beautif...


Q3 - 3. Write a python program to access the search bar and search button on images.google.com and 
scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [80]:
import selenium
import pandas as pd
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException

In [89]:
url = 'https://images.google.com/'
driver.get(url)

In [90]:
search_fruits = driver.find_element_by_xpath('//input[@title="Search"]')
search_fruits.send_keys('Fruits')

In [91]:
search_btn = driver.find_element_by_xpath('//button[@class="Tg7LZd"]')
search_btn.click()

In [92]:
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

In [93]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

199

In [94]:
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open(r"Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

In [95]:
url = 'https://images.google.com/'
driver.get(url)

In [96]:
search_cars = driver.find_element_by_xpath('//input[@title="Search"]')
search_cars.send_keys('cars')

In [97]:
search_btn = driver.find_element_by_xpath('//button[@class="Tg7LZd"]')
search_btn.click()

In [98]:
car_images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

In [99]:
car_img_urls = []
car_img_data = []
for car_image in car_images:
    source= car_image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            car_img_urls.append(source)
len(car_img_urls)

199

In [100]:
for i in range(len(car_img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(car_img_urls[i])
    file = open(r"Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

In [104]:
url = 'https://images.google.com/'
driver.get(url)

In [105]:
search_ML = driver.find_element_by_xpath('//input[@title="Search"]')
search_ML.send_keys('Machine Learning')

In [106]:
search_btn = driver.find_element_by_xpath('//button[@class="Tg7LZd"]')
search_btn.click()

In [107]:
ML_images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

In [108]:
ML_img_urls = []
ML_img_data = []
for ML_image in ML_images:
    source= ML_image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            ML_img_urls.append(source)
len(ML_img_urls)

175

In [109]:
for i in range(len(ML_img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(ML_img_urls[i])
    file = open(r"Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

In [110]:
Images=pd.DataFrame({})
Images['img_urls']=img_urls[0:100]
Images['car_img_urls']=car_img_urls[0:100]
Images['ML_img_urls']=ML_img_urls[0:100]
Images

,img_urls,car_img_urls,ML_img_urls
0,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
1,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
3,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
...,...,...,...
95,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
96,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
97,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
98,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...


Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on 
www.flipkart.com and scrape following details for all the search results displayed on 1st page. 
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, 
“Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display 
Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. 
Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe 
and CSV.

In [119]:
import selenium
import pandas as pd
from selenium import webdriver
import time
import os
import io
import re
import requests
from PIL import Image
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

In [120]:
driver= webdriver.Chrome(r'C:\Users\Admin\Downloads\chromedriver_win32 (2)\chromedriver.exe')
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="f7be5a3aff075a1edb50a510e6e087a3")>

In [127]:
url='https://www.flipkart.com/'
driver.get(url)

In [128]:
search_product = driver.find_element_by_xpath("//input[@class='_3704LK']")
search_product.send_keys('smartphones')

In [129]:
search_btn=driver.find_element_by_xpath("//button[@class='L0Z3Pu']")
search_btn.click()

In [130]:
brandurl=driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')

In [131]:
brandurl

[<selenium.webdriver.remote.webelement.WebElement (session="f7be5a3aff075a1edb50a510e6e087a3", element="badd072b-5b19-41a1-b4f5-d5d52a141120")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f7be5a3aff075a1edb50a510e6e087a3", element="95299126-3e88-41c3-a2e1-934d1af70f2e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f7be5a3aff075a1edb50a510e6e087a3", element="7889faac-f83a-4f9a-9754-4d076b532d81")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f7be5a3aff075a1edb50a510e6e087a3", element="fed55c74-c030-4b4d-a862-89b7ff5ae521")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f7be5a3aff075a1edb50a510e6e087a3", element="0274ce7e-10f6-4c99-8a3e-3b667c40ea5c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f7be5a3aff075a1edb50a510e6e087a3", element="53306d57-3989-4e47-9bca-b9d0c439ef90")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f7be5a3aff075a1edb50a510e6e087a3", element="f9f9d572-c2bd-43ec-b6e8-59

In [132]:
urls=[]
brand=[]
colour=[]
ram=[]
rom=[]
pcam=[]
scam=[]
dsize=[]
dres=[]
proc=[]
procor=[]
bat=[]
price=[]

In [133]:
for i in brandurl:
    urls.append(i.get_attribute("href"))

In [134]:
for i in urls:
    driver.get(i)
    brandname=driver.find_element_by_xpath("//span[@class='B_NuCI']")
    brand.append(brandname.text)
    read_btn=driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
    read_btn.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[@class='B_NuCI']"}
  (Session info: chrome=94.0.4606.71)


5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [136]:
url='https://www.google.com/maps/'
driver.get(url)

City = input('Enter City Name : ')
driver.find_element_by_xpath("//input[@class='tactile-searchbox-input']").send_keys(City)
driver.find_element_by_xpath("//button[@class='searchbox-searchbutton']").click()

latitude=driver.current_url.split('/@')[1].split(',')[0]
longtitude=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]
print('Latitude:',latitude,'Longitude:',longtitude)

Enter City Name : Pune
Latitude: 18.5508662 Longitude: 73.9450864


6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 –
September 20) from trak.in.

In [137]:
driver= webdriver.Chrome(r'C:\Users\Admin\Downloads\chromedriver_win32 (2)\chromedriver.exe')
driver.get('https://trak.in/')
time.sleep(2)

driver.get(driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href'))

In [140]:
Date = []
Startup_Name = []
Industry_Vertical = []
Sub_Vertical = []
Location = []
Investor = []
Investment_Type = []
Amount_in_usd = []

In [141]:
#scrapping for date
for i in driver.find_elements_by_xpath('//td[@class="column-2"]'):
    date.append(i.text)

In [142]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        Date.append(d.text)
 # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        Startup_Name.append(n.text)
    
    # Industry_Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        Industry_Vertical.append(n.text)
    
    # Sub_Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        Sub_Vertical.append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        Location.append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        Investor.append(n.text)
    
    # Investment_Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        Investment_Type.append(n.text)
    
    # Amount_in_usd
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        Amount_in_usd.append(a.text)

In [143]:
print(Date)
print(Startup_Name) 
print(Industry_Vertical) 
print(Sub_Vertical) 
print(Location) 
print(Investor)  
print(Investment_Type)
print(Amount_in_usd)

['15/07/2020', '16/07/2020', '16/07/2020', '14/07/2020', '13/07/2020', '09/07/2020', '10/07/2020', '15/07/2020', '14/07/2020', '13/07/2020', '15/08/2020', '13/08/2020', '13/08/2020', '04/08/2020', '11/08/2020', '12/08/2020', '14/08/2020', '10/08/2020', '07/08/2020', '13/08/2020', '03/08/2020', '04/08/2020', '08/09/2020', '12/09/2020', '09/09/2020', '02/09/2020', '09/09/2020', '07/09/2020', '07/09/2020', '31/08/2020', '31/08/2020', '03/09/2020', '08/09/2020']
['Flipkart', 'Vedantu', 'Crio', 'goDutch', 'Mystifly', 'JetSynthesys', 'gigIndia', 'PumPumPum', 'FLYX', 'Open Appliances Pvt. Ltd.', 'Practo', 'Medlife', 'HungerBox', 'Dunzo', 'Terra.do', 'Classplus', 'Niyo', 'ZestMoney', 'FreshToHome', 'Eduvanz', 'CrowdPouch', 'DrinkPrime', 'Byju’s', 'mCaffeine', 'Qshala', 'Winzo', 'Hippo Video', 'Melorra', '1mg', 'mfine', 'Apna', 'Railofy', 'Cell Propulsion']
['E-commerce', 'EduTech', 'EduTech', 'FinTech', 'Airfare Marketplace', 'Gaming and Entertainment', 'Marketplace', 'Automotive Rental', 'OTT

In [144]:
Funding_Deals=pd.DataFrame({})
Funding_Deals['Date']=Date
Funding_Deals['Startup Name']=Startup_Name
Funding_Deals['Location']=Location
Funding_Deals['Industry Vertical']=Industry_Vertical
Funding_Deals['Sub Vertical']=Sub_Vertical
Funding_Deals['Investor']=Investor
Funding_Deals['Investment Type']=Investment_Type
Funding_Deals['Amount In_USD']=Amount_in_usd

In [145]:
Funding_Deals

,Date,Startup Name,Location,Industry Vertical,Sub Vertical,Investor,Investment Type,Amount In_USD
0,15/07/2020,Flipkart,Bangalore,E-commerce,E-commerce,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,Bangalore,EduTech,Online Tutoring,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,Bangalore,EduTech,Learning Platform for Developers,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,Mumbai,FinTech,Group Payments,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Singapore and Bangalore,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Pune,Gaming and Entertainment,Gaming and Entertainment,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Pune,Marketplace,"Crowd Sourcing, Freelance",Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Gurgaon,Automotive Rental,Used Car-leasing platform,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,New York and Delhi,OTT Player,Streaming Social Network,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Bangalore,Information Technology,Internet-of-Things Security Solutions,Unicorn India Ventures,Venture-Series Unknown,"500,000"


7. Write a program to scrap all the available details of best gaming laptops from digit.in. 

In [147]:
url='https://www.digit.in/top-products/best-gaming-laptops-40.html'
driver.get(url)

In [148]:
brand=[]
OS=[]
gpr=[]
rom=[]
dsize=[]
proc=[]
price=[]
wgt=[]
dms=[]

In [149]:
prices=driver.find_elements_by_xpath("//td[@class='smprice']")
for i in prices:
    price.append(i.text)
        
brandname=driver.find_elements_by_xpath("//div[@class='right-container']//h3")
for i in brandname:
    brand.append(i.text)
    
os=driver.find_elements_by_xpath("//td[text()='OS']/following-sibling::td[2]")
for i in os:
    OS.append(i.text)

ds=driver.find_elements_by_xpath("//td[text()='Display']/following-sibling::td[2]")
for i in ds:
    dsize.append(i.text)

pro=driver.find_elements_by_xpath("//td[text()='Processor']/following-sibling::td[2]")
for i in pro:
    proc.append(i.text)

    
roms=driver.find_elements_by_xpath("//td[text()='Memory']/following-sibling::td[2]")
for i in roms:
    rom.append(i.text)

wg=driver.find_elements_by_xpath("//td[text()='Weight']/following-sibling::td[2]")
for i in wg:
    wgt.append(i.text)

dm=driver.find_elements_by_xpath("//td[text()='Dimension']/following-sibling::td[2]")
for i in dm:
    dms.append(i.text)

gp=driver.find_elements_by_xpath("//td[text()='Graphics Processor']/following-sibling::td[2]")
for i in gp:
    gpr.append(i.text)

In [150]:
print(brand)
print(OS)
print(dms)
print(rom)
print(gpr)
print(dsize)
print(proc)
print(price)
print(wgt)

['ALIENWARE AREA 51M R2', 'ALIENWARE M15 R3', 'ASUS ROG STRIX SCAR 15', 'ASUS ROG ZEPHYRUS G14', 'LENOVO LEGION 5I', 'ASUS ROG ZEPHYRUS DUO 15', 'ACER ASPIRE 7 GAMING']
['Windows 10 Home', 'Windows 10 Home', 'Windows 10 Home', 'Windows 10 Home', 'Windows 10 Pro', 'Windows 10', 'Windows 10 Home']
['27.65 x 402.6 x 319.14', 'NA', '35.4 x 25.9 x 2.26', '32.5 x 22.1 x 1.8', '363.06 x 259.61 x 23.57', '268.30 x 360.00 x 20.90', '2.29 x 36.3 x 25.4']
['1 TB SSD/16 GBGB DDR4', '1 TB SSD/16 GBGB DDR4', '1 TB SSD/16 GBGB DDR4', '1 TB SSD/16 GBGB DDR4', '1 TB SSD/16 GBGB DDR4', '512 GB SSD/4 GBGB DDR4', '512 GB SSD/8 GBGB DDR4']
['Intel® UHD Graphics 630', 'NA', 'NVIDIA® GeForce RTX™ 3070', 'NVIDIA GeForce RTX 2060', 'NVIDIA® GeForce® GTX 1650 4GB', 'NVIDIA GeForce RTX 2070 Max-Q', 'NVIDIA® GeForce® GTX 1650']
['17.3" (1920 x 1080)', '15.6" (3840 x 2160)', '15.6" (1920 x 1080)', '14" (1920 x 1080)', '15.6" (1920 x 1080)', '15.6" (3840 x 1100)', '15.6" (1920 x 1080)']
['10th Generation Intel® Cor

In [151]:
Gaming_Laps=pd.DataFrame({})
Gaming_Laps['Product Name']=brand
Gaming_Laps['OS']=OS
Gaming_Laps['Dimension']=dms
Gaming_Laps['Storage']=rom
Gaming_Laps['Graphics Processor']=gpr
Gaming_Laps['Display Size']=dsize
Gaming_Laps['Processor']=proc
Gaming_Laps['Price']=price
Gaming_Laps['Weight']=wgt

In [152]:
Gaming_Laps

,Product Name,OS,Dimension,Storage,Graphics Processor,Display Size,Processor,Price,Weight
0,ALIENWARE AREA 51M R2,Windows 10 Home,27.65 x 402.6 x 319.14,1 TB SSD/16 GBGB DDR4,Intel® UHD Graphics 630,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,N/A,4.1
1,ALIENWARE M15 R3,Windows 10 Home,NA,1 TB SSD/16 GBGB DDR4,NA,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,"₹ 319,990",NA
2,ASUS ROG STRIX SCAR 15,Windows 10 Home,35.4 x 25.9 x 2.26,1 TB SSD/16 GBGB DDR4,NVIDIA® GeForce RTX™ 3070,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,"₹ 215,990",2.30
3,ASUS ROG ZEPHYRUS G14,Windows 10 Home,32.5 x 22.1 x 1.8,1 TB SSD/16 GBGB DDR4,NVIDIA GeForce RTX 2060,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,"₹ 164,990",1.65
4,LENOVO LEGION 5I,Windows 10 Pro,363.06 x 259.61 x 23.57,1 TB SSD/16 GBGB DDR4,NVIDIA® GeForce® GTX 1650 4GB,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,"₹ 76,988",2.3
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,268.30 x 360.00 x 20.90,512 GB SSD/4 GBGB DDR4,NVIDIA GeForce RTX 2070 Max-Q,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,"₹ 185,000",2.4
6,ACER ASPIRE 7 GAMING,Windows 10 Home,2.29 x 36.3 x 25.4,512 GB SSD/8 GBGB DDR4,NVIDIA® GeForce® GTX 1650,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,"₹ 56,990",2.15


8. Write a python program to scrape the details for all billionaires from www.forbes.com. 
Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, 
“Industry”.

In [158]:
driver= webdriver.Chrome(r'C:\Users\Admin\Downloads\chromedriver_win32 (2)\chromedriver.exe')

time.sleep(3)

driver.get('https://www.forbes.com')

In [159]:
driver.find_element_by_xpath("//button[@class='icon--hamburger']").click() # clicking on the hamburget button

#getting the billionaire page
billionaire=driver.find_element_by_xpath("//li[@class='header__channel header__color--centennial-silver header__hoverable']/a")
driver.get(billionaire.get_attribute('href'))
time.sleep(2)

driver.find_element_by_xpath("//h2[@class='chansec-special-feature__title no-label--special-feature']").click()
time.sleep(2)

#creating the empty list
Rank=[]
Name=[]
Net_worth=[]
Age=[]
Citizenship=[]
Source=[]
Industry=[]

#there are 15 pages hence we will scrap through range function
start=0
end=15
for page in range(start,end):
    #scrapping for rank
    rank=driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rank:
        Rank.append(i.text)

    #scrapping for name
    name=driver.find_elements_by_xpath("//div[@class='personName']")
    for i in name:
        Name.append(i.text)

    #scrapping for Networth
    networth=driver.find_elements_by_xpath("//div[@class='netWorth']/div")
    for i in networth:
        Net_worth.append(i.text)
    
    #scrapping for age
    age=driver.find_elements_by_xpath("//div[@class='age']/div")
    for i in age:
        Age.append(i.text)
    
    #scrapping for citizenship
    citi=driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in citi:
        Citizenship.append(i.text)

    #scrapping for source
    source=driver.find_elements_by_xpath("//div[@class='source-column']/div[1]")
    for i in source:
        Source.append(i.text)
    
    #scrapping for industry
    industry=driver.find_elements_by_xpath("//div[@class='category']/div")
    for i in industry:
        Industry.append(i.text)
    
    #to go on the next page
    try:
        nxt=driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']").click()
    except:
        NoSuchElementException

In [160]:
Forbes_Billionarie={'Rank':Rank,'Name':Name,'Net Worth':Net_worth,'Age':Age,
                    'Citizenship':Citizenship,'Source':Source,'Industry':Industry}
Forbes_Billionarie=pd.DataFrame(data=Forbes_Billionarie)
Forbes_Billionarie

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2905,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2906,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2907,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2908,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


9. Write a program to extract at least 500 Comments, Comment upvote and time when comment 
was posted from any YouTube Video.

In [169]:
driver= webdriver.Chrome(r'C:\Users\Admin\Downloads\chromedriver_win32 (2)\chromedriver.exe')

In [177]:
#creating the empty list
Comments=[]
Upvote=[]
Updated=[]

#loading the drivers and url
driver= webdriver.Chrome(r'C:\Users\Admin\Downloads\chromedriver_win32 (2)\chromedriver.exe')
wait = WebDriverWait(driver,15)
driver.get("https://www.youtube.com/watch?v=ukzFI9rgwfU")

#creating the program to scroll down the comments and let it load
SCROLL_PAUSE_TIME = 2
CYCLES = 100

html = driver.find_element_by_tag_name('html')
html.send_keys(Keys.PAGE_DOWN)  
html.send_keys(Keys.PAGE_DOWN)  
time.sleep(SCROLL_PAUSE_TIME * 3)

for i in range(CYCLES):
    html.send_keys(Keys.END)
    time.sleep(SCROLL_PAUSE_TIME)

#scrapping for comments
comment = driver.find_elements_by_xpath('//*[@id="content-text"]')
for i in comment:
    Comments.append(i.text)

#scrapping for upvote
upvote = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
for i in upvote:
    Upvote.append(i.text)

#scrapping for updated on
updated=driver.find_elements_by_xpath('//*[@id="header-author"]/yt-formatted-string/a')
for i in updated:
    Updated.append(i.text)

#creating the dataframe
Youtube={'Comments':Comments,'Comment Upvote':Upvote, 'Updated on':Updated}
Youtube=pd.DataFrame(data=Youtube)
Youtube

,Comments,Comment Upvote,Updated on
0,🔥Explore Our FREE Courses With Completion Cert...,8,1 month ago
1,**Summary**:\n- machine learning is the genera...,2.8K,2 years ago
2,Scenario 1: Facebook recognizes your friend in...,1K,2 years ago
3,I'm impressed by the way you taught. Teacher s...,231,1 year ago
4,Literally learnt more from you than 4 years in...,131,4 months ago
...,...,...,...
335,i like your video.....\ni learnt a lot abt ml....,1,2 years ago
336,"YouTube learning from what we watch , what we ...",2,2 years ago
337,"first one is supervised, second is unsupervise...",,2 years ago
338,Just slow down the speaking speed to 0.75 .,,1 year ago


Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [180]:
url = "https://www.hostelworld.com/"
driver.get(url)

In [182]:
search_city=driver.find_element_by_xpath("//input[@id='search-input-field']")
search_city.send_keys('London')

In [183]:
driver.find_element_by_xpath('//*[@id="predicted-search-results"]/li[2]').click()
driver.find_element_by_id('search-button').click()

In [184]:
names=driver.find_elements_by_xpath("//h2[@class='title title-6']")
name=[]
for i in names:
    name.append(i.text)
name

['PubLove @ The Green Man, Paddington',
 'Mornington Camden',
 "St Christopher's Inn - Liverpool Street",
 'London Waterloo Hostel',
 "St Christopher's Camden",
 'Smart Camden Inn Hostel',
 'PubLove @ The Crown, Battersea',
 "St Christopher's Inn - London Bridge",
 'Barmy Badger Backpackers',
 'Clink261',
 'Smart Russell Square Hostel',
 'Mapesbury Hostel',
 "St Christopher's Village",
 "St Christopher's Oasis-Female Only",
 "St Christopher's Shepherds Bush",
 'Astor Museum Inn',
 'Astor Kensington',
 'New Cross Inn Hostel',
 'Hootananny Hostel',
 'Astor Hyde Park',
 'Phoenix Hostel',
 '247london Hostel and Private Rooms',
 'Burlington Residence',
 "Wombat's The City Hostel London",
 'Venture Hostel',
 'Urbany Hostel London',
 'Prime Backpackers Angel',
 'Generator London',
 'SoHostel',
 'Pickwick Hall']

In [185]:
dist=driver.find_elements_by_xpath("//span[@class='description']")
distnc=[]
for i in dist:
    distnc.append(i.text)
distnc

['Hostel - 4.3km from city centre',
 'Hostel - 4.1km from city centre',
 'Hostel - 3.2km from city centre',
 'Hostel - 0.7km from city centre',
 'Hostel - 4.3km from city centre',
 'Hostel - 4.4km from city centre',
 'Hostel - 4.7km from city centre',
 'Hostel - 1.8km from city centre',
 'Hostel - 5.5km from city centre',
 'Hostel - 3.2km from city centre',
 'Hostel - 2.6km from city centre',
 'Hostel - 8.2km from city centre',
 'Hostel - 1.8km from city centre',
 'Hostel - 1.8km from city centre',
 'Hostel - 7km from city centre',
 'Hostel - 2.2km from city centre',
 'Hostel - 4.9km from city centre',
 'Hostel - 6.1km from city centre',
 'Hostel - 5km from city centre',
 'Hostel - 4.3km from city centre',
 'Hostel - 4.2km from city centre',
 'Hostel - 8.2km from city centre',
 'Hostel - 7.1km from city centre',
 'Hostel - 3.6km from city centre',
 'Hostel - 6.9km from city centre',
 'Hostel - 5.4km from city centre',
 'Hostel - 3.6km from city centre',
 'Hostel - 3km from city centre'

In [186]:
ovralrev=driver.find_elements_by_xpath("//div[@class='keyword']")
overallrev=[]
for i in ovralrev:
    overallrev.append(i.text)
overallrev

['Superb',
 'Fabulous',
 'Good',
 'Superb',
 'Superb',
 'Very Good',
 'Fabulous',
 'Fabulous',
 'Good',
 'Good',
 'Superb',
 'Very Good',
 'Good',
 'Good',
 'Fabulous',
 'Fabulous',
 'Very Good',
 'Superb',
 'Very Good',
 'Good',
 'Fabulous',
 'Superb',
 'Very Good',
 'Good',
 'Superb',
 'Fabulous',
 'Rating',
 'Superb',
 'Superb',
 'Very Good',
 'Fabulous',
 'Superb']

In [187]:
totalrev = driver.find_elements_by_xpath("//div[@class='reviews']")
total_reviews=[]
for i in totalrev:
    total_reviews.append(i.text)
total_reviews

['394 Total Reviews',
 '41 Total Reviews',
 '357 Total Reviews',
 '2420 Total Reviews',
 '3711 Total Reviews',
 '2691 Total Reviews',
 '214 Total Reviews',
 '3237 Total Reviews',
 '1661 Total Reviews',
 '3182 Total Reviews',
 '9516 Total Reviews',
 '357 Total Reviews',
 '10859 Total Reviews',
 '598 Total Reviews',
 '627 Total Reviews',
 '8736 Total Reviews',
 '6330 Total Reviews',
 '1018 Total Reviews',
 '1410 Total Reviews',
 '11343 Total Reviews',
 '3435 Total Reviews',
 '443 Total Reviews',
 '30 Total Reviews',
 '13204 Total Reviews',
 '1170 Total Reviews',
 '127 Total Reviews',
 '515 Total Reviews',
 '6749 Total Reviews',
 '3994 Total Reviews',
 '2234 Total Reviews']

In [188]:
rtngs=driver.find_elements_by_xpath("//div[@class='score orange big']")
ratings=[]
for i in rtngs:
    ratings.append(i.text)

In [189]:
dorms_pvt_price=driver.find_elements_by_xpath("//div[@class='price-col']")
dorms_pvt_prices=[]
for i in dorms_pvt_price:
    dorms_pvt_prices.append(i.text.replace('\n',''))
dorms_pvt_prices

['Privates FromRs14477',
 'Dorms FromRs1609',
 'No Privates Available',
 'Dorms FromRs3152',
 'Privates FromRs9517',
 'Dorms FromRs1699',
 'Privates FromRs4614',
 'Dorms FromRs1538',
 'Privates FromRs7171',
 'Dorms FromRs1247',
 'Privates FromRs5200',
 'Dorms FromRs1285',
 'Privates FromRs6334',
 'Dorms FromRs1609',
 'Privates FromRs6300',
 'Dorms FromRs1280',
 'Privates FromRs6032',
 'Dorms FromRs2111',
 'No Privates Available',
 'Dorms FromRs1330',
 'No Privates Available',
 'Dorms FromRs1352',
 'Privates FromRs4524',
 'No Dorms Available',
 'Privates FromRs7641',
 'Dorms FromRs1196',
 'No Privates Available',
 'Dorms FromRs1297',
 'Privates FromRs4826',
 'Dorms FromRs1056',
 'No Privates Available',
 'Dorms FromRs1609',
 'No Privates Available',
 'Dorms FromRs1407',
 'Privates FromRs3210',
 'Dorms FromRs1203',
 'No Privates Available',
 'Dorms FromRs1624',
 'No Privates Available',
 'Dorms FromRs1508',
 'Privates FromRs3045',
 'Dorms FromRs1200',
 'Privates FromRs5127',
 'No Dorms A

In [190]:
pvt = []
for i in range(0,len(dorms_pvt_prices),2):
    pvt.append(dorms_pvt_prices[i])
pvt

['Privates FromRs14477',
 'No Privates Available',
 'Privates FromRs9517',
 'Privates FromRs4614',
 'Privates FromRs7171',
 'Privates FromRs5200',
 'Privates FromRs6334',
 'Privates FromRs6300',
 'Privates FromRs6032',
 'No Privates Available',
 'No Privates Available',
 'Privates FromRs4524',
 'Privates FromRs7641',
 'No Privates Available',
 'Privates FromRs4826',
 'No Privates Available',
 'No Privates Available',
 'Privates FromRs3210',
 'No Privates Available',
 'No Privates Available',
 'Privates FromRs3045',
 'Privates FromRs5127',
 'Privates FromRs8043',
 'Privates FromRs8738',
 'Privates FromRs5807',
 'Privates FromRs5157',
 'Privates FromRs7365',
 'Privates FromRs8182',
 'No Privates Available',
 'Privates FromRs5027']

In [191]:
dorms = []
for i in range(1,len(dorms_pvt_prices),2):
    dorms.append(dorms_pvt_prices[i])
dorms

['Dorms FromRs1609',
 'Dorms FromRs3152',
 'Dorms FromRs1699',
 'Dorms FromRs1538',
 'Dorms FromRs1247',
 'Dorms FromRs1285',
 'Dorms FromRs1609',
 'Dorms FromRs1280',
 'Dorms FromRs2111',
 'Dorms FromRs1330',
 'Dorms FromRs1352',
 'No Dorms Available',
 'Dorms FromRs1196',
 'Dorms FromRs1297',
 'Dorms FromRs1056',
 'Dorms FromRs1609',
 'Dorms FromRs1407',
 'Dorms FromRs1203',
 'Dorms FromRs1624',
 'Dorms FromRs1508',
 'Dorms FromRs1200',
 'No Dorms Available',
 'Dorms FromRs2051',
 'Dorms FromRs1878',
 'Dorms FromRs1121',
 'Dorms FromRs1749',
 'Dorms FromRs2379',
 'Dorms FromRs1949',
 'Dorms FromRs2011',
 'No Dorms Available']

In [192]:
facilitiess=driver.find_elements_by_xpath("//div[@class='has-wifi']")
facilityy=[]
for i in facilitiess:
    facilityy.append(i.text)
facilityy

['Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi',
 'Free WiFi']

In [193]:
pptdesc=driver.find_elements_by_xpath("//div[@class='cleanliness']")
prptdesc=[]
for i in pptdesc:
    prptdesc.append(i.text)
prptdesc

['Awesome Cleanliness',
 'Fantastic Cleanliness',
 'Fantastic Cleanliness',
 'Awesome Cleanliness',
 'Excellent Cleanliness',
 'Fantastic Cleanliness',
 'Excellent Cleanliness',
 'Fantastic Cleanliness',
 'Awesome Cleanliness',
 'Awesome Cleanliness',
 'Fantastic Cleanliness',
 'Excellent Cleanliness',
 'Excellent Cleanliness',
 'Fantastic Cleanliness',
 'Awesome Cleanliness',
 'Awesome Cleanliness',
 'Good Cleanliness',
 'Fantastic Cleanliness',
 'Excellent Cleanliness',
 'Excellent Cleanliness',
 'Fantastic Cleanliness',
 'Fantastic Cleanliness',
 'Fantastic Cleanliness',
 'Fantastic Cleanliness',
 'Excellent Cleanliness',
 'Excellent Cleanliness',
 'Excellent Cleanliness']

In [196]:
hostels=pd.DataFrame()
hostels["Hostel_Name"]=name[:27]
hostels["City_Distance"]=distnc[:27]
hostels["Overall_Review"]=overallrev[:27]
hostels["Total_Reviews"]=total_reviews[:27]
hostels["ratings"]=ratings[:27]
hostels["Privates_From_Price"]=pvt[:27]
hostels["Dorms_From_Price"]=dorms[:27]
hostels["facilities"]=facilityy[:27]
hostels["property description"]=prptdesc[:27]

In [197]:
hostels

,Hostel_Name,City_Distance,Overall_Review,Total_Reviews,ratings,Privates_From_Price,Dorms_From_Price,facilities,property description
0,"PubLove @ The Green Man, Paddington",Hostel - 4.3km from city centre,Superb,394 Total Reviews,6.2,Privates FromRs14477,Dorms FromRs1609,Free WiFi,Awesome Cleanliness
1,Mornington Camden,Hostel - 4.1km from city centre,Fabulous,41 Total Reviews,10,No Privates Available,Dorms FromRs3152,Free WiFi,Fantastic Cleanliness
2,St Christopher's Inn - Liverpool Street,Hostel - 3.2km from city centre,Good,357 Total Reviews,10,Privates FromRs9517,Dorms FromRs1699,Free WiFi,Fantastic Cleanliness
3,London Waterloo Hostel,Hostel - 0.7km from city centre,Superb,2420 Total Reviews,7.2,Privates FromRs4614,Dorms FromRs1538,Free WiFi,Awesome Cleanliness
4,St Christopher's Camden,Hostel - 4.3km from city centre,Superb,3711 Total Reviews,8.2,Privates FromRs7171,Dorms FromRs1247,Free WiFi,Excellent Cleanliness
5,Smart Camden Inn Hostel,Hostel - 4.4km from city centre,Very Good,2691 Total Reviews,8.6,Privates FromRs5200,Dorms FromRs1285,Free WiFi,Fantastic Cleanliness
6,"PubLove @ The Crown, Battersea",Hostel - 4.7km from city centre,Fabulous,214 Total Reviews,6.8,Privates FromRs6334,Dorms FromRs1609,Free WiFi,Excellent Cleanliness
7,St Christopher's Inn - London Bridge,Hostel - 1.8km from city centre,Fabulous,3237 Total Reviews,6.5,Privates FromRs6300,Dorms FromRs1280,Free WiFi,Fantastic Cleanliness
8,Barmy Badger Backpackers,Hostel - 5.5km from city centre,Good,1661 Total Reviews,9.6,Privates FromRs6032,Dorms FromRs2111,Free WiFi,Awesome Cleanliness
9,Clink261,Hostel - 3.2km from city centre,Good,3182 Total Reviews,7.1,No Privates Available,Dorms FromRs1330,Free WiFi,Awesome Cleanliness
